# Modeling and derived metrics

With synthetic raw signals in place we can illustrate how Monumental Labs might normalize them with DuckDB, compute production KPIs, and build a lightweight predictive model for finishing time.

In [ ]:
from __future__ import annotations

from pathlib import Path

import duckdb
import numpy as np
import pandas as pd

DATA_DIR = Path("../data")
con = duckdb.connect(database=":memory:")

In [ ]:
con.execute(
    f"""
    CREATE OR REPLACE TABLE jobs AS
    SELECT * FROM read_csv_auto('{(DATA_DIR / 'jobs.csv').as_posix()}', header=True);
    """
)
con.execute(
    f"""
    CREATE OR REPLACE TABLE toolpaths AS
    SELECT * FROM read_csv_auto('{(DATA_DIR / 'toolpaths.csv').as_posix()}', header=True);
    """
)
con.execute(
    f"""
    CREATE OR REPLACE TABLE telemetry AS
    SELECT * FROM read_csv_auto('{(DATA_DIR / 'telemetry.csv').as_posix()}', header=True);
    """
)

con.execute(
    """
    CREATE OR REPLACE TABLE job_contact AS
    SELECT job_id, SUM(contact_time_s) AS total_contact_s
    FROM toolpaths
    GROUP BY 1;
    """
)

### Toolpath efficiency

We estimate the volume removed per toolpath by allocating each job's total volume based on that toolpath's share of contact time.

In [ ]:
con.execute(
    """
    CREATE OR REPLACE TABLE toolpath_metrics AS
    WITH contact_share AS (
        SELECT t.*, j.volume_removed_cm3, jc.total_contact_s,
               CASE WHEN jc.total_contact_s = 0 THEN 0
                    ELSE t.contact_time_s / jc.total_contact_s END AS share
        FROM toolpaths t
        JOIN job_contact jc USING (job_id)
        JOIN jobs j USING (job_id)
    )
    SELECT
        toolpath_id,
        job_id,
        feed_mm_min,
        rpm,
        spindle_current_a,
        contact_time_s,
        tool_id,
        share,
        share * volume_removed_cm3 AS estimated_volume_cm3,
        CASE WHEN contact_time_s = 0 THEN NULL
             ELSE (share * volume_removed_cm3) / contact_time_s END AS toolpath_efficiency_cm3_s
    FROM contact_share;
    """
)

toolpath_metrics = con.execute(
    "SELECT * FROM toolpath_metrics ORDER BY job_id, toolpath_id LIMIT 5"
).df()
toolpath_metrics

### Tool health view

A crude wear index is the sum of spindle current multiplied by cutting time.

In [ ]:
tool_health = con.execute(
    """
    SELECT
        tool_id,
        SUM(spindle_current_a * contact_time_s) AS tool_wear_index,
        SUM(contact_time_s) AS total_contact_s,
        AVG(spindle_current_a) AS avg_current_a
    FROM toolpaths
    GROUP BY 1
    ORDER BY tool_wear_index DESC;
    """
).df()
tool_health

### Job-level metrics

We consolidate job performance to feed the dashboard.

In [ ]:
job_metrics = con.execute(
    """
    WITH efficiency AS (
        SELECT job_id, AVG(toolpath_efficiency_cm3_s) AS avg_toolpath_efficiency_cm3_s
        FROM toolpath_metrics
        GROUP BY 1
    )
    SELECT
        j.job_id,
        j.material,
        j.geometry_complexity,
        j.volume_removed_cm3,
        j.finish_minutes,
        j.quoted_price_usd,
        j.scheduled_start,
        jc.total_contact_s,
        j.volume_removed_cm3 / NULLIF(jc.total_contact_s / 60.0, 0) AS removal_rate_cm3_min,
        e.avg_toolpath_efficiency_cm3_s
    FROM jobs j
    JOIN job_contact jc USING (job_id)
    JOIN efficiency e USING (job_id)
    ORDER BY j.job_id;
    """
).df()
job_metrics.head()

### Predictive finishing time

We fit a simple linear regression on material, complexity, and average toolpath efficiency to estimate finishing time. The synthetic data keeps the math deterministic yet realistic.

In [ ]:
materials = pd.get_dummies(job_metrics["material"], prefix="mat")
features = pd.concat(
    [
        materials,
        job_metrics[["geometry_complexity", "avg_toolpath_efficiency_cm3_s", "removal_rate_cm3_min"]],
    ],
    axis=1,
)
X = features.to_numpy()
y = job_metrics["finish_minutes"].to_numpy()
X_with_bias = np.hstack([np.ones((X.shape[0], 1)), X])
coeffs, *_ = np.linalg.lstsq(X_with_bias, y, rcond=None)
pred = X_with_bias @ coeffs

job_predictions = job_metrics.copy()
job_predictions["pred_finish_minutes"] = np.round(pred, 1)
job_predictions["residual_minutes"] = job_predictions["finish_minutes"] - job_predictions["pred_finish_minutes"]
rmse = np.sqrt(np.mean(np.square(job_predictions["residual_minutes"])) )
rmse

In [ ]:
job_predictions[["job_id", "material", "finish_minutes", "pred_finish_minutes", "residual_minutes"]].head()

The resulting tables — `jobs`, `toolpaths`, `telemetry`, `toolpath_metrics`, `tool_health`, and `job_predictions` — feed directly into the Streamlit dashboard to highlight throughput, economics, and maintenance signals.